<a href="https://colab.research.google.com/github/RajashreeDahal4/Looks-Like-Grains/blob/main/Looks_Like_Grains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tensorflow==2.4.1

In [ ]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, Reshape, Concatenate,MaxPooling2D, AveragePooling2D,Input, BatchNormalization, Activation, UpSampling2D, Concatenate, LeakyReLU,Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D,Multiply,SpatialDropout2D,Add,Input,Dropout
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow_datasets as tfds
from tensorflow.keras.applications.densenet import preprocess_input


In [ ]:
# model.summary()
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
IMG_SIZE = 224
LBL = dict(zip(['B_BSMUT1', 'B_CLEV5B', 'B_DISTO', 'B_GRMEND', 'B_HDBARL',
       'B_PICKLD', 'B_SKINED', 'B_SOUND', 'B_SPRTED', 'B_SPTMLD',
       'O_GROAT', 'O_HDOATS', 'O_SEPAFF', 'O_SOUND', 'O_SPOTMA',
       'WD_RADPODS', 'WD_RYEGRASS', 'WD_SPEARGRASS', 'WD_WILDOATS',
       'W_DISTO', 'W_FLDFUN', 'W_INSDA2', 'W_PICKLE', 'W_SEVERE',
       'W_SOUND', 'W_SPROUT', 'W_STAIND', 'W_WHITEG'], range(28)))
cls_map = dict(zip(LBL.values(),LBL.keys()))

In [ ]:
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [ ]:
def augmentation(x):
   x=tf.keras.preprocessing.image.random_rotation(x,30,row_axis=0,col_axis=1,channel_axis=2)#,channel_axis=2
   x=tf.image.random_flip_left_right(x)
   x=tf.keras.preprocessing.image.random_zoom(x,zoom_range=[0.9,0.9],row_axis=0,col_axis=1,channel_axis=2) #channel_axis=2
   x=tf.keras.preprocessing.image.random_shift(x, wrg=0.1, hrg=0.1,row_axis=1,col_axis=1,channel_axis=2)
   return x

In [ ]:
def tf_augmentation(image, label):
  im_shape = image.shape
  [image,] = tf.py_function(augmentation, [image], [tf.float32])
  image.set_shape(im_shape)
  return image, label

In [ ]:
# def augmentation(x,y):
#   x=tf.keras.preprocessing.image.random_rotation(x,30)  #,channel_axis=2
#   x=tf.image.random_flip_left_right(x)
#   # x=tf.keras.preprocessing.image.random_zoom(x,zoom_range=[0.3,0.3],channel_axis=2) #channel_axis=2
#   x=tf.keras.preprocessing.image.random_shift(x, wrg=0.1, hrg=0.1)
#   x=tf.keras.preprocessing.image.random_shear(x,3)  # afaile socheko hai 3 ko value
#   x=tf.keras.layers.experimental.preprocessing.RandomFlip(x,'HORIZONTAL')
#   return x,y

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
directory1 = '/content/drive/MyDrive/LOOK into grain /Train'
directory2 = '/content/drive/MyDrive/LOOK into grain /Valid'
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory1, labels='inferred', label_mode='categorical',
    class_names=LBL.keys(), color_mode='rgb', batch_size=16, image_size=(224,
    224), shuffle=True, seed=None, validation_split=0, subset=None,
    interpolation='bilinear', follow_links=False)
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory2, labels='inferred', label_mode='categorical',
    class_names=LBL.keys(), color_mode='rgb', batch_size=16, image_size=(224,
    224), shuffle=True, seed=0, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False)
train_dataset= train_dataset.unbatch()

train_dataset = train_dataset.map(tf_augmentation)
train_dataset = train_dataset.batch(batch_size=16, drop_remainder=True)
# train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
# train_dataset= train_dataset.shuffle(1000)
train_dataset=train_dataset.repeat()
test_dataset =valid_dataset.repeat()

Found 19182 files belonging to 28 classes.
Found 7648 files belonging to 28 classes.


In [ ]:
from tensorflow.keras.callbacks import CSVLogger ,LearningRateScheduler,Callback
class CustomLearningRateScheduler(tf.keras.callbacks.Callback):
    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule
    def on_epoch_end(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        scheduled_lr = self.schedule(epoch, lr)
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nbatch %05d: Learning rate is %8f." % (epoch, scheduled_lr))
def lr_scheduler(epoch,lr):
      
    #   if (i+13+12+6)> 30:
    #     lr =0.00000025347102
    #     return lr
    lr = lr/1.25
    print(lr)
    return lr
filepath = '/content/drive/MyDrive/classification/'
Checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_accuracy', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
)

In [ ]:
NUM_CLASSES =28
from tensorflow.keras.applications.efficientnet import EfficientNetB6
input = Input((224,224,3))
# input = tf.keras.layers.experimental.preprocessing.Rescaling(scale =1/255)(input)
# model2 = EfficientNetB6(input_tensor=input,weights ='imagenet' ,include_top=False)
# print(model2.summary())
# model2.trainable=False
# x = GlobalAveragePooling2D()(model2.output)
# x = BatchNormalization()(x)
# top_dropout_rate = 0.2
# x = Dropout(top_dropout_rate, name="top_dropout")(x)
# # outputs = Dense(1024, activation="relu"")(x)
# # x = BatchNormalization()(x)
# # x = Dropout(top_dropout_rate)(x)
# outputs = Dense(NUM_CLASSES, activation="softmax", name="pred")(x)
# model = Model(input, outputs, name="EfficientNet")

In [ ]:
input = Input((224,224,3))
model2 = tf.keras.applications.efficientnet.EfficientNetB6(
    include_top=False, weights='imagenet', input_tensor=input
)
print(model2.summary())
# model2 = tf.keras.applications.densenet.DenseNet201(
#     include_top=False, weights='imagenet', input_tensor=input
# )
model2.trainable=False
names =['block2a_expand_activation','block2b_expand_activation','block2f_activation','block3b_expand_activation','block3f_activation','block4a_expand_activation','block4d_activation','block4g_activation','block6a_expand_activation','block5c_activation','block5h_expand_activation','block6f_activation','block6k_activation','top_activation']


for i,name in enumerate(names):
    outputs = model2.get_layer(name).output
    print(outputs.shape)
    x = GlobalAveragePooling2D()(outputs)
    x = BatchNormalization()(x)
    if i==0:
        y=x
    else:
        y=Concatenate()([x,y])
        print(y.shape)
top_dropout_rate = 0.5
# x = BatchNormalization()(y)
x = Dropout(top_dropout_rate, name="top_dropout")(y)
outputs = Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

model =Model(input,outputs)
model.summary()

In [ ]:
# model=tf.keras.models.load_model('/content/drive/MyDrive/classification')

In [ ]:
metrics = tf.keras.metrics.Accuracy(name='accuracy')
def custom_metrics(y_true,y_pred):
  # print(y_true)
  def weightage(j):
    if j in [7,13,24]:
      return 2.98
    return 0.004
  NUM_CLASSES=28
  accuracy=[]
  for i in range(28):
    y_pred1 = tf.math.argmax(y_pred, axis=-1)
    # print(y_pred1)
    y_pred1 = tf.one_hot(y_pred1,NUM_CLASSES,axis =-1)
    # print(y_pred1)
    y_pred1 = y_pred1[:,i]
    # print(y_pred1)
    # y_pred1 = tf.reshape(y_pred1,[-1,NUM_CLASSES])
    y_true1 = y_true[:,i]
    weig=weightage(i)
    acc = metrics(y_true1,y_pred1,sample_weight=weig)
    accuracy.append(acc)
  # print("Checking the shape of accuracy")
  # print("NOw reducing the sum")
  accuracy=tf.reduce_sum(accuracy)/28
  return accuracy

In [ ]:
train_steps = 19182//16
valid_steps = 7648//16
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy",custom_metrics])
model.fit(train_dataset,epochs =20,steps_per_epoch=train_steps,validation_data=test_dataset,validation_steps=valid_steps,callbacks=[Checkpoint])

Epoch 1/20
1198/1198 [==============================] - 8636s 7s/step - loss: 1.7324 - accuracy: 0.5592 - custom_metrics: 0.8411 - val_loss: 7.6086 - val_accuracy: 0.2501 - val_custom_metrics: 0.8431


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/drive/MyDrive/classification/assets
Epoch 2/20
1198/1198 [==============================] - 4241s 4s/step - loss: 1.1639 - accuracy: 0.6813 - custom_metrics: 0.8429 - val_loss: 8.4241 - val_accuracy: 0.2308 - val_custom_metrics: 0.8480
Epoch 3/20
 198/1198 [===>..........................] - ETA: 57:51 - loss: 1.0485 - accuracy: 0.7058 - custom_metrics: 0.8412

In [ ]:
model =Model(input,outputs)#this was the best
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = DenseNet201(include_top=False, input_tensor=inputs, weights="imagenet")
    model.trainable = False
    names =['pool2_relu','pool3_conv','pool4_relu','relu']
    for i,name in enumerate(names):
        output = model.get_layer(name).output
        x = layers.GlobalAveragePooling2D()(output)
        if i==0:
            y=x
        else:
            y=layers.Concatenate()([x,y])
    top_dropout_rate = 0.2
    x  = layers.BatchNormalization()(y)
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)
    model = tf.keras.Model(inputs, outputs, name="DenseNet201")

In [ ]:
train_steps = 19182//16
valid_steps = 7648//16
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_dataset,epochs =10,steps_per_epoch=train_steps,validation_data=test_dataset,validation_steps=valid_steps,callbacks=[Checkpoint])

In [ ]:
model2 = tf.keras.applications.efficientnet.EfficientNetB6(
    include_top=False, weights='imagenet', input_tensor=input
)
# print(model2.summary())
# model2 = tf.keras.applications.densenet.DenseNet201(
#     include_top=False, weights='imagenet', input_tensor=input
# )
model2.trainable=False
names =['block2a_expand_activation','block3a_expand_activation','block4a_expand_activation','block6a_expand_activation','block6k_activation','top_activation']
for i,name in enumerate(names):
    outputs = model2.get_layer(name).output
    x = GlobalAveragePooling2D()(outputs)
    x = BatchNormalization()(x)
    top_dropout_rate = 0.2
    x = Dropout(top_dropout_rate)(x)
    x= Dense(NUM_CLASSES, activation="softmax")(x)
    x =Reshape((NUM_CLASSES,1))(x)
    if i==0:
        y=x
    else:
      y=Concatenate()([x,y])
y=tf.keras.layers.Lambda(lambda x:(tf.reduce_sum(x,axis=2))/3.0)(y)
y=tf.keras.layers.Flatten()(y)
model =Model(input,outputs=y)

model.summary()

In [ ]:
train_steps = 19182//16
valid_steps = 7648//16
ptimizer = tf.keras.optimizers.Adam(learning_rate=.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_dataset,epochs =10,steps_per_epoch=train_steps,validation_data=test_dataset,validation_steps=valid_steps,callbacks=[Checkpoint])

In [ ]:
model2 = tf.keras.applications.efficientnet.EfficientNetB6(
    include_top=False, weights='imagenet', input_tensor=input
)
model2.trainable =True
x = GlobalAveragePooling2D()(model2.output)
top_dropout_rate = 0.2
x = Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = Dense(NUM_CLASSES, activation="softmax", name="pred")(x)
model =Model(input,outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=.0004)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_dataset,epochs =3,steps_per_epoch=train_steps,validation_data=valid_dataset,validation_steps=valid_steps,callbacks=[Checkpoint] )
optimizer = tf.keras.optimizers.SGD(learning_rate=.005,momentum=0.9)
model.fit(train_dataset,epochs =5,steps_per_epoch=train_steps,validation_data=valid_dataset,validation_steps=valid_steps,callbacks=[Checkpoint] )

In [ ]:
import cv2 as cv
img = cv.imread('/content/drive/MyDrive/LOOK into grain /Train/B_DISTO/0a24670e-1c33-49d5-a45d-1facfb743a7a.png')

In [ ]:
plt.imshow(img)

In [ ]:
model.save('/content/drive/MyDrive/classification/',save_format='tf')
